<a href="https://colab.research.google.com/github/ecordeiro/cefet/blob/main/busca_aleatoria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random

In [2]:
class Individual:
    def __init__(self, chromosome_length):
        # Inicializa um indivíduo com um cromossomo aleatório
        self.chromosome = [random.random() for _ in range(chromosome_length)]
        self.fitness = None
        self.dominated_solutions = []  # Soluções dominadas por este indivíduo
        self.domination_count = 0  # Contagem de quantos indivíduos dominam este indivíduo

def gerar_populacao(num_individuos, num_variaveis, limites):
    """
    Gera uma população de indivíduos com valores aleatórios para cada variável dentro dos limites especificados.

    Argumentos:
    num_individuos (int): Número de indivíduos na população.
    num_variaveis (int): Número de variáveis para cada indivíduo.
    limites (list): Lista de tuplas contendo os limites para cada variável. Cada tupla deve conter o limite inferior e superior.

    Retorna:
    list: Uma lista de listas, onde cada lista representa um indivíduo com valores aleatórios para cada variável.
    """
    populacao = []
    for _ in range(num_individuos):
        individuo = []
        for i in range(num_variaveis):
            limite_inferior, limite_superior = limites
            valor = np.random.uniform(limite_inferior, limite_superior)
            individuo.append(valor)
        populacao.append(individuo)
    return populacao

    population = [Individual(chromosome_length) for _ in range(population_size)]


def f1(x_i):
    n = len(x_i)
    valor = sum([(elemento**2 - 4) - 10 * np.cos(2 * np.pi * (elemento**2 - 4)) for elemento in x_i])
    return 10 * n + valor

def f2(x_i):
    n = len(x_i)
    valor = sum([(elemento**2 - 10 * np.cos(2 * np.pi * elemento)) for elemento in x_i])
    return 10 * n + valor

In [3]:
num_individuos = 200
num_variaveis = 2
limites = (-5.12, 5.12)

populacao = gerar_populacao(num_individuos, num_variaveis, limites)

In [4]:
# Exemplo de uso:
num_individuos = 50
num_variaveis = 2
limites = (-5.12, 5.12)

nova_geracao = gerar_populacao(num_individuos, num_variaveis, limites)

for indice, elemento in enumerate(nova_geracao):
    print((indice,f1(elemento),f2(elemento)))

(0, 10.244515605793456, 15.150528556338362)
(1, 18.913777449987386, 45.990143968979936)
(2, -6.722306022105421, 16.974780213900914)
(3, 6.708605475858221, 6.396137156017289)
(4, 40.69678198434287, 15.048499333917105)
(5, 20.834074565495875, 35.04089280302265)
(6, 37.52815134470698, 30.876883665963295)
(7, 16.57147229692886, 11.112905487855937)
(8, 39.87246924312963, 49.05246306016559)
(9, 29.70480758678856, 19.53691344215076)
(10, 23.916268339503294, 40.308397855779994)
(11, 38.3212332812415, 37.768840057884915)
(12, 18.913836361731562, 9.312970645170514)
(13, 33.66294100819695, 23.02483454851067)
(14, 46.81261589806735, 45.14996265901435)
(15, 17.92743100163457, 16.25964998337885)
(16, 31.858355683056967, 30.214999720617158)
(17, 13.094989718869071, 33.57019446237511)
(18, 17.82172668366057, 34.66565394925691)
(19, 12.452537186634538, 7.615859240495183)
(20, 22.868451885116922, 41.98824894274834)
(21, 5.393772715151851, 20.334661507293568)
(22, 47.68829725405652, 62.81615424163779)
(2

In [ ]:
import random

class Individual:
    def __init__(self, chromosome_length):
        # Inicializa um indivíduo com um cromossomo aleatório
        self.chromosome = [random.random() for _ in range(chromosome_length)]
        self.fitness = None
        self.dominated_solutions = []  # Soluções dominadas por este indivíduo
        self.domination_count = 0  # Contagem de quantos indivíduos dominam este indivíduo

def crowding_distance_assignment(population):
    # Zera as listas de soluções dominadas e contagens de dominação
    for ind in population:
        ind.dominated_solutions = []

    # Calcula as soluções dominadas e atualiza a contagem de dominação para cada indivíduo
    for i in range(len(population)):
        for j in range(i + 1, len(population)):
            if dominates(population[i].fitness, population[j].fitness):
                population[j].dominated_solutions.append(i)
                population[i].domination_count += 1
            elif dominates(population[j].fitness, population[i].fitness):
                population[i].dominated_solutions.append(j)
                population[j].domination_count += 1

    # Calcula a distância de multidão de cada indivíduo
    for ind in population:
        ind.crowding_distance = 0

    num_objectives = len(population[0].fitness)
    # Calcula a distância de multidão para cada objetivo
    for m in range(num_objectives):
        population.sort(key=lambda x: x.fitness[m])
        population[0].crowding_distance = float('inf')
        population[-1].crowding_distance = float('inf')
        for i in range(1, len(population) - 1):
            population[i].crowding_distance += (population[i + 1].fitness[m] - population[i - 1].fitness[m])

def dominates(fitness1, fitness2):
    # Verifica se fitness1 domina fitness2
    return all(fitness1[i] <= fitness2[i] for i in range(len(fitness1))) and any(fitness1[i] < fitness2[i] for i in range(len(fitness1)))

def nsga2(population, num_generations, tournament_size, crossover_rate, mutation_rate):
    # Executa o algoritmo NSGA-II para um número fixo de gerações
    for _ in range(num_generations):
        offspring = []
        # Cria a nova população de descendentes
        while len(offspring) < len(population):
            parent1 = tournament_selection(population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)
            child1, child2 = crossover(parent1, parent2, crossover_rate)
            offspring.extend(mutate(child1, mutation_rate))
            offspring.extend(mutate(child2, mutation_rate))
        population.extend(offspring)
        crowding_distance_assignment(population)
        population.sort(key=lambda x: (x.domination_count, -x.crowding_distance))
        population = population[:len(population) // 2]  # Seleciona os melhores indivíduos

def tournament_selection(population, size):
    # Seleciona um indivíduo do torneio
    tournament = random.sample(population, size)
    tournament.sort(key=lambda x: (x.domination_count, -x.crowding_distance))
    return tournament[0]

def crossover(parent1, parent2, rate):
    # Executa o crossover entre dois pais com uma determinada taxa de crossover
    if random.random() < rate:
        point = random.randint(1, len(parent1.chromosome) - 1)
        child1 = Individual(len(parent1.chromosome))
        child2 = Individual(len(parent2.chromosome))
        child1.chromosome = parent1.chromosome[:point] + parent2.chromosome[point:]
        child2.chromosome = parent2.chromosome[:point] + parent1.chromosome[point:]
        return child1, child2
    else:
        return parent1, parent2

def mutate(individual, rate):
    # Realiza a mutação em um indivíduo com uma determinada taxa de mutação
    mutated_individuals = []
    for i in range(len(individual.chromosome)):
        if random.random() < rate:
            mutated_individual = Individual(len(individual.chromosome))
            mutated_individual.chromosome = individual.chromosome[:]
            mutated_individual.chromosome[i] = random.random()
            mutated_individuals.append(mutated_individual)
    return mutated_individuals

# Exemplo de uso
population_size = 100
chromosome_length = 10
num_generations = 50
tournament_size = 5
crossover_rate = 0.8
mutation_rate = 0.01

population = [Individual(chromosome_length) for _ in range(population_size)]
nsga2(population, num_generations, tournament_size, crossover_rate, mutation_rate)

# Após a execução do NSGA-II, você pode acessar as soluções não dominadas da população
non_dominated_solutions = [ind for ind in population if ind.domination_count == 0]
print("Número de soluções não dominadas:", len(non_dominated_solutions))
